# Analysis of Data from Chronic Worry Study 

## Basic study structure:

1. Learn transition probabilities by pressing "1" or "0" and seeing how often each action leads to certain images. Before participant sees the image an action leads to (one image 60% of the time, another image 40% of the time), words of different valences are flashed on screen for 500 msec. Participants are instructed to ignore these words.


2. 30 second rest


3. Test period where two amounts of money are placed next to each image. Participant is asked to plan which action will get to the image with the most money.

10 conditions (2 rounds for each condition): 

Test phase: 20 probes, each with varying amounts of money associated with each image.

### Get basic info on subjects


In [2]:
import pandas as pd
import numpy as np

worry_data=pd.read_csv('questionnaire_data.csv')
print('# Males={}'.format(np.sum(1*(worry_data['Response']=='Male'))))
print('# Females={}'.format(np.sum(1*(worry_data['Response']=='Female'))))

# Males=65
# Females=70


## Get Worry Questionnaire data

In [43]:
import pandas as pd
import numpy as np

#set up variables
PSWQ_Scores=[]
id_col='Participant External Session ID'
reverse_scored=[1,3,8,10,11]
rs_dict={1:5,2:4,3:3,4:2,5:1}
current_score=0
reverse_score_counter=0
counter=0
IDs=['Holder']
sc=0
#get row locations where PSWQ items are
ilocs=[]

#check if they faithfully filled out the questionnaire
rs_items=[]
non_rs_items=[]
sub=1
c_id=1
for row,data in worry_data.iterrows():
   
    if str(worry_data['Question Key'][row])=='BEGIN QUESTIONNAIRE' or row==19854:
        sc+=1
        if str(worry_data[id_col][row])!=IDs[c_id-1]:
            IDs.append(str(worry_data[id_col][row]))
            c_id+=1
    
        PSWQ_Scores.append(current_score)
        current_score=0
        reverse_score_counter=0
        counter=0
#         print('\n Reverse_Score_Check for Subject {}'.format(sub))
#         print('RS avg: {}'.format(np.mean(rs_items)))
#         print('NON RS avg: {}'.format(np.mean(non_rs_items)))
#         print('\n')
        sub+=1
        rs_items=[]
        non_rs_items=[]

    
    elif str(worry_data['Question Key'][row]).startswith('ps') and not str(worry_data['Question Key'][row]).endswith('quantised'):
        if len(worry_data['Question Key'][row])>5:
            if int(worry_data['Question Key'][row][-2:]) in reverse_scored:
                rs_items.append(int(worry_data['Response'][row]))
                current_score+=rs_dict[int(worry_data['Response'][row])]
                counter+=1
                reverse_score_counter+=1
                ilocs.append(row)
            else:
                non_rs_items.append(int(worry_data['Response'][row]))
                current_score+=int(worry_data['Response'][row])
                counter+=1
                ilocs.append(row)
        else:
            if int(worry_data['Question Key'][row][-1]) in reverse_scored:
                rs_items.append(int(worry_data['Response'][row]))
                current_score+=rs_dict[int(worry_data['Response'][row])]
                counter+=1
                reverse_score_counter+=1
                ilocs.append(row)
            else:
                current_score+=int(worry_data['Response'][row])
                non_rs_items.append(int(worry_data['Response'][row]))
                counter+=1
                ilocs.append(row)
            
PSWQ_Scores.remove(0)
#np.save('PSWQ_Scores',PSWQ_Scores)
print(PSWQ_Scores)
print('Mean on PSWQ =\n {}'.format(np.mean(PSWQ_Scores)))
print('\n')

print('SD on PSWQ =\n {}'.format(np.std(PSWQ_Scores)))
print('\n')


print('# of subs = {}'.format(len(PSWQ_Scores)))

del IDs[0]
print(len(IDs))
print(IDs)


print('\n {}'.format(sc))

[37, 51, 46, 60, 66, 61, 59, 69, 67, 72, 72, 31, 48, 39, 42, 57, 65, 64, 67, 60, 50, 57, 55, 79, 57, 39, 64, 60, 34, 58, 60, 41, 24, 70, 79, 50, 71, 39, 63, 63, 37, 46, 56, 46, 41, 70, 75, 66, 62, 79, 59, 60, 62, 71, 76, 26, 46, 66, 48, 80, 55, 38, 34, 44, 66, 65, 72, 64, 31, 71, 39, 57, 64, 51, 62, 52, 68, 38, 55, 62, 55, 74, 78, 51, 53, 41, 39, 52, 39, 60, 56, 51, 32, 53, 33, 65, 78, 57, 58, 27, 47, 55, 46, 53, 42, 53, 49, 57, 52, 77, 55, 56, 36, 65, 71, 66, 38, 32, 54, 20, 65, 66, 55, 73, 30, 57, 57, 70, 43, 44, 58, 67, 61, 52, 79, 38]
Mean on PSWQ =
 55.125


SD on PSWQ =
 13.650812031437804


# of subs = 136
136
['5daf141d8df32d001210acc2', '5daf1579b6f30f0011a5ed27', '5daf15aecc409a0014d4fdd3', '5daf157119813900165f53f3', '5daf16707d039a00179099cd', '5daf1438028c550017706eaf', '5daf179d41fbb6001660d0a9', '5daf17c07ac8810017839036', '5daf17ff6726310013bf7d57', '5daf1818952fb10014a92e65', '5daf18b3a540ef0016289f48', '5daf177ba257250016932664', '5daf197447977500168924f6', '5daf19495

In [50]:
#save self-reported state mood taken prior to task
sr_mood=df_sr_currentmood['Response'].values
#np.save('selfreportmood_r2',sr_mood)

In [53]:
import pandas as pd
import numpy as np
from scipy import stats
#load  in data and just consider columsn of importance for analysis
part_order=IDs
df_task1=pd.read_csv('task_v1.csv')
df_task2=pd.read_csv('task_v2.csv')

df_task_v1=df_task1[['Participant External Session ID','display','forced_choice','Response','image2','test_image1', 
                   'test_image2','test_image1_value', 
                   'test_image2_value']]

df_task_v2=df_task2[['Participant External Session ID','display','forced_choice','Response','image2','test_image1', 
                   'test_image2','test_image1_value', 
                   'test_image2_value']]

# Define best-action dictionary for present design
# best action per condition (6 conditions)
best_answer_key_v1={'neutral_1': [[0, 'Drink 1.jpg'], [1, 'Timber 4.jpg']], 
                    'rare_positive_1': [[0, 'Acorns 2.jpg'], [1, 'Bark 5.jpg']], 
                    'neutral_2': [[0, 'Bricks 1.jpg'], [1, 'Fence 2.jpg']], 
                    'common_positive_1': [[0, 'Cold 2.jpg'], [1, 'Cups 3.jpg']], 
                    'rare_threat_1': [[0, 'Office supplies 4.jpg'], [1, 'Pumpkin 1.jpg']],
                    'common_threat_1': [[0, 'Power lines 1.jpg'], [1, 'Dirt 3.jpg']], 
                    'fifty_half_threat': [[0, 'Paper 5.jpg'], [1, 'Cotton swabs 3.jpg']],
                    'half_rare_threat_1': [[0, 'Keys 1.jpg'], [1, 'Storage 2.jpg']], 
                    'qt_neutral_2': [[0, 'Grass 6.jpg'], [1, 'Billiards 1.jpg']], 
                    'qt_rare_threat_1': [[0, 'Cups 1.jpg'], [1, 'Storage 1.jpg']]}


best_answer_key_v2={'neutral_1': [[0, 'Phone 1.jpg'], [1, 'Soccer 1.jpg']], 
                    'rare_positive_1': [[0, 'Keys 1.jpg'], [1, 'Fire hydrant 1.jpg']], 
                    'neutral_2': [[0, 'Billiards 1.jpg'], [1, 'Bubble 1.jpg']], 
                    'common_positive_1': [[0, 'Bricks 1.jpg'], [1, 'Acorns 2.jpg']], 
                    'rare_threat_1': [[0, 'Paper 5.jpg'], [1, 'Office supplies 1.jpg']], 
                    'common_threat_1': [[0, 'Office supplies 4.jpg'], [1, 'Fence 2.jpg']],
                    'fifty_half_threat': [[0, 'Cold 2.jpg'], [1, 'Cold 2.jpg']], 
                    'half_rare_threat_1': [[0, 'Snow 3.jpg'], [1, 'Storage 1.jpg']], 
                    'qt_neutral_2': [[0, 'Crosswalk 1.jpg'], [1, 'Crow 1.jpg']], 
                    'qt_rare_threat_1': [[0, 'Paintbrush 1.jpg'], [1, 'Grass 1.jpg']]}

#best answers per condition: lists
rt1=[]
rp1=[]
cp1=[]
n1=[]
n2=[]
qrt=[]
hrt=[]
qn=[]
fif=[]
ct1=[]

all_sub_IDs=[]
value_differential=[]
temp_row=[]
bas=[]
appended_c=0
test_counter=1
tally=0
invalid_scores={'NaN'}
counter=0
conditions=[]
start_new_test_set=0
start_new_subject=0
best_action_tally=0
condition_counter=0
sub_counter=0
current_subject=1

#subject specific data
rt_sub=[]
ct_sub=[]
rp_sub=[]
cp_sub=[]
neut1_sub=[]
neut2_sub=[]
qrt_sub=[]
qn_sub=[]
hrt_sub=[]
fif_sub=[]
current_choice_data=[]

choice_data_3d=np.zeros((136,9,20)) #to be populated below
for version in range(2):
    if version ==0:
        final_row=120362
        df_task_r=df_task_v1
    else:
        final_row=117972
        df_task_r=df_task_v2
#     print(df_task_r['Participant External Session ID'])
    for row,data in df_task_r.iterrows():
    
        if str(df_task_r['display'][row]).startswith('test'):
            if counter==0:
                conditions.append(df_task_r['display'][row][5:])
                if version==0:
                    condition_info=best_answer_key_v1[conditions[counter]]
                else:
                    condition_info=best_answer_key_v2[conditions[counter]]
                    
                new_condition=0
                counter+=1
            elif df_task_r['display'][row][5:]!=conditions[counter-1]:
                        conditions.append(df_task_r['display'][row][5:])
                        if version==0:
                            condition_info=best_answer_key_v1[conditions[counter]]
                        else:
                            condition_info=best_answer_key_v2[conditions[counter]]
                            
                        
                        if conditions[counter-1]=='rare_threat_1':
                            rt1.append(best_action_tally)
                            rt_sub=current_choice_data


                        elif conditions[counter-1]=='rare_positive_1':
                            rp1.append(best_action_tally)
                            rp_sub=current_choice_data


                        elif conditions[counter-1]=='common_positive_1':
                            cp1.append(best_action_tally)
                            cp_sub=current_choice_data

                        elif conditions[counter-1]=='neutral_1':
                            n1.append(best_action_tally)
                            neut1_sub=current_choice_data

                        elif conditions[counter-1]=='neutral_2':
                            n2.append(best_action_tally)
                            neut2_sub=current_choice_data
                        
                        #new conditions
                        elif conditions[counter-1]=='common_threat_1':
                            ct1.append(best_action_tally)
                            ct_sub=current_choice_data
                            
                        elif conditions[counter-1]=='half_rare_threat_1':
                            hrt.append(best_action_tally)
                            hrt_sub=current_choice_data
                            
                        elif conditions[counter-1]=='fifty_half_threat':
                            fif.append(best_action_tally)
                            fif_sub=current_choice_data
                            
                        elif conditions[counter-1]=='qt_rare_threat_1':
                            qrt.append(best_action_tally)
                            qrt_sub=current_choice_data
                            
                        elif conditions[counter-1]=='qt_neutral_2':
                            qn.append(best_action_tally)
                            qn_sub=current_choice_data
                        
                    
                        
                        if sub_counter==135:
                            print(conditions[counter-1])
                            
                        counter+=1
                        best_action_tally=0
                        current_choice_data=[]
                        condition_counter+=1
                        #after 10 blocks, new subject

                        if condition_counter==10:
                            current_subject+=1

                            if len(neut1_sub)<20:
                                neut1_sub=[int(x) for x in neut1_sub+np.zeros(20-len(neut1_sub)).tolist()]


                            if len(neut2_sub)<20:
                                neut2_sub=[int(x) for x in neut2_sub+np.zeros(20-len(neut2_sub)).tolist()]


                            if len(rt_sub)<20:
                                rt_sub=[int(x) for x in rt_sub+np.zeros(20-len(rt_sub)).tolist()]

                            if len(rp_sub)<20:
                                rp_sub=[int(x) for x in rp_sub+np.zeros(20-len(rp_sub)).tolist()]


                            if len(cp_sub)<20:
                                cp_sub=[int(x) for x in cp_sub+np.zeros(20-len(cp_sub)).tolist()]
                            
                            if len(ct_sub)<20:
                                ct_sub=[int(x) for x in ct_sub+np.zeros(20-len(ct_sub)).tolist()]
                            if len(qn_sub)<20:
                                qn_sub=[int(x) for x in qn_sub+np.zeros(20-len(qn_sub)).tolist()]
                            if len(qrt_sub)<20:
                                qrt_sub=[int(x) for x in qrt_sub+np.zeros(20-len(qrt_sub)).tolist()]
                            if len(hrt_sub)<20:
                                hrt_sub=[int(x) for x in hrt_sub+np.zeros(20-len(hrt_sub)).tolist()]
                            if len(fif_sub)<20:
                                fif_sub=[int(x) for x in fif_sub+np.zeros(20-len(fif_sub)).tolist()]
                            


                            #simulate two equal rounds
                            choice_data_3d[sub_counter,0]=neut2_sub
                            choice_data_3d[sub_counter,1]=cp_sub
                            choice_data_3d[sub_counter,2]=ct_sub
                            choice_data_3d[sub_counter,3]=rp_sub
                            choice_data_3d[sub_counter,4]=rt_sub
                            choice_data_3d[sub_counter,5]=hrt_sub
                            choice_data_3d[sub_counter,6]=qrt_sub
                            choice_data_3d[sub_counter,7]=qn_sub
                            choice_data_3d[sub_counter,8]=fif_sub
                            
                            all_sub_IDs.append(df_task_r['Participant External Session ID'][row])
                            sub_counter+=1
                            print(df_task_r['Participant External Session ID'][row-5])
                            print(sub_counter)
                            if sub_counter==136:
                                print('lastROW: {}'.format(row))
                            condition_counter=0

                            rt_sub=[]
                            ct_sub=[]
                            rp_sub=[]
                            cp_sub=[]
                            neut1_sub=[]
                            neut2_sub=[]
                            qrt_sub=[]
                            qn_sub=[]
                            hrt_sub=[]
                            fif_sub=[]

            else:
                new_condition=0

            #Get values and convert from strings to floating point
            value1=df_task_r['test_image1_value'][row]
            if "p" in value1:
                if value1.startswith('2'):
                    value1=float(value1[0:2])*.01
                else:   
                    value1=float(value1[0:4])

            else:
                if value1.startswith('£'):
                    value1=float(value1[1])
                else:
                    value1=float(value1[0])


            value2=df_task_r['test_image2_value'][row]
            if "p" in value2:
                if value2.startswith('2'):
                    value2=float(value2[0:2])*.01
                else:   
                    value2=float(value2[0:4])
            else:
                if value2.startswith('£'):
                    value2=float(value2[1])
                else:
                    value2=float(value2[0])


            if value1>value2:

                best_option=df_task_r['test_image1'][row]
            else:
                best_option=df_task_r['test_image2'][row]



            #get response and convert to integer
            try:
                current_response=int(df_task_r['Response'][row])
            except:
                current_response='missing'




            # determine if participant made best choice
            for info_total in condition_info:
                for info in info_total:
                    if best_option == str(info):
                        best_action=info_total[0]
            if current_subject==1:
                if conditions[counter-1]=='neutral_1':
                    bas.append(best_action)

            #for last subject only that doesn't meet the condition above for indexing
            if row==final_row:
                if version==0:
                    rt1.append(best_action_tally)
                    rt_sub=current_choice_data
                    all_sub_IDs.append(df_task_r['Participant External Session ID'][row])
                    sub_counter+=1
                    print(df_task_r['Participant External Session ID'][row-5])
                    print(sub_counter)
                    
                else:
                    cp1.append(best_action_tally)
                    cp_sub=current_choice_data
                    print('lastSUB')


                if len(neut1_sub)<20:
                    neut1_sub=[int(x) for x in neut1_sub+np.zeros(20-len(neut1_sub)).tolist()]

                if len(neut2_sub)<20:
                    neut2_sub=[int(x) for x in neut2_sub+np.zeros(20-len(neut2_sub)).tolist()]

                if len(rt_sub)<20:
                    rt_sub=[int(x) for x in rt_sub+np.zeros(20-len(rt_sub)).tolist()]

                if len(rp_sub)<20:
                    rp_sub=[int(x) for x in rp_sub+np.zeros(20-len(rp_sub)).tolist()]

                if len(cp_sub)<20:
                    cp_sub=[int(x) for x in cp_sub+np.zeros(20-len(cp_sub)).tolist()]
                    
                if len(ct_sub)<20:
                    ct_sub=[int(x) for x in ct_sub+np.zeros(20-len(ct_sub)).tolist()]
                if len(qn_sub)<20:
                    qn_sub=[int(x) for x in qn_sub+np.zeros(20-len(qn_sub)).tolist()]
                if len(qrt_sub)<20:
                    qrt_sub=[int(x) for x in qrt_sub+np.zeros(20-len(qrt_sub)).tolist()]
                if len(hrt_sub)<20:
                    hrt_sub=[int(x) for x in hrt_sub+np.zeros(20-len(hrt_sub)).tolist()]
                if len(fif_sub)<20:
                    fif_sub=[int(x) for x in fif_sub+np.zeros(20-len(fif_sub)).tolist()]



                choice_data_3d[sub_counter,0]=neut2_sub
                choice_data_3d[sub_counter,1]=cp_sub
                choice_data_3d[sub_counter,2]=ct_sub
                choice_data_3d[sub_counter,3]=rp_sub
                choice_data_3d[sub_counter,4]=rt_sub
                choice_data_3d[sub_counter,5]=hrt_sub
                choice_data_3d[sub_counter,6]=qrt_sub
                choice_data_3d[sub_counter,7]=qn_sub
                choice_data_3d[sub_counter,8]=fif_sub



                condition_counter=0

                rt_sub=[]
                ct_sub=[]
                rp_sub=[]
                cp_sub=[]
                neut1_sub=[]
                neut2_sub=[]
                qrt_sub=[]
                qn_sub=[]
                hrt_sub=[]
                fif_sub=[]

            else:

                if current_response==best_action:
                    best_action_tally+=1
                    current_choice_data.append(1.0)
                    if current_subject==1:
                        if test_counter<21:
                            test_counter+=1
                            temp_row.append(np.abs(value1-value2))
                        else:
                            value_differential.append(temp_row)
                            temp_row=[]
                            test_counter=2
                            temp_row.append(np.abs(value1-value2))
                elif current_response=='missing':
                    x='missing'
                else:
                    current_choice_data.append(0.0)
                    if current_subject==1:
                        if test_counter<21:
                            test_counter+=1
                            temp_row.append(np.abs(value1-value2))
                        else:
                            value_differential.append(temp_row)
                            temp_row=[]
                            test_counter=2
                            temp_row.append(np.abs(value1-value2))

value_differential.append(temp_row)

# print('rt1 {}'.format(np.mean(rt1)))
# print('rp1 {}'.format(np.mean(rp1)))
# print('cp1 {}'.format(np.mean(cp1)))
# print('ct1 {}'.format(np.mean(ct1)))
# print('n1 {}'.format(np.mean(n1)))
# print('n2 {}'.format(np.mean(n2)))
# print('qrt {}'.format(np.mean(qrt)))
# print('hrt {}'.format(np.mean(hrt)))
# print('qn {}'.format(np.mean(qn)))
# print('fif {}'.format(np.mean(fif)))
print('\n')
print('#Subjects:')
print(len(all_sub_IDs))
print('\n')
bad_subs=[]
counter=0
for x in n2:
    if x<10:
        print('bc: {}, counter:{}'.format(x,counter))
        bad_subs.append(counter)
    counter+=1
        

# #convert to numpy arrays    
# rt1=np.array(rt1)
# print(rt1)
# rp1=np.array(rp1)
# print(rp1)
# cp1=np.array(cp1)
# print(cp1)
# n1=np.array(n1)
# print(n1)
# print(conditions[0:9])
# print(np.mean(value_differential))
# print(value_differential)
# vd=[y-0.8 for x in value_differential for y in x]
# print(vd)

choice_data_3d = choice_data_3d.astype(int)
print(choice_data_3d[1,:,:])
#np.save('data_pilot2',choice_data_3d)
#np.save('value_diff_atchoice_r2_demeaned',vd)


5daf15aecc409a0014d4fdd3
1
5daf18b3a540ef0016289f48
2
5daf1438028c550017706eaf
3
5daf17c07ac8810017839036
4
5daf1a4f5bd1b30013605d81
5
5daf1afbab64fb00142c57b4
6
5daf1c4458a2e500170f1e37
7
5daf197447977500168924f6
8
5daf1e2318dc520011b2638e
9
5daf1dd2bc5196001a762cad
10
5daf20689611830014dd44f2
11
5daf214389da2b0014973567
12
5daf23bfa257250016932685
13
5daf254a8484bc0018bd0f45
14
5daf1acae28dd200163e005a
15
5daf2871a2cae4001738f022
16
5daf2bd8a075d100161fc6e4
17
5daf2de73990ed00185648cc
18
5daf2e86f663bd0018ee11ed
19
5daf326088f16400151bfd94
20
5daf3528eafbbc00141cdf47
21
5daf37c2e28dd200163e00b2
22
5daf3bf4b3844500109a019f
23
5daf3e772b35810016d8d989
24
5daf3ddfaebbbb0017ff85c6
25
5daf410e19813900125f5295
26
5daf4448d3f1e60014988473
27
5daf501a37b2ce0015a586e7
28
5daf3feb4e440c0014783b6d
29
5daf513371db9b0017e7fecd
30
5daf561ecc409a0015d4fda4
31
5daf5d6bd876770018e504bc
32
5daf609a2b35810012d8d788
33
5daf6366476c3f0016caae08
34
5daf641b79f9ac00112761e0
35
5daf6b792da657001445b75c
36
5

In [59]:
print(choice_data_3d[68,:,:])


[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [13]:

print('rt1 {}'.format(np.mean(rt1)))
print('rp1 {}'.format(np.mean(rp1)))
print('cp1 {}'.format(np.mean(cp1)))
print('ct1 {}'.format(np.mean(ct1)))
print('n1 {}'.format(np.mean(n1)))
print('n2 {}'.format(np.mean(n2)))
print('qrt {}'.format(np.mean(qrt)))
print('hrt {}'.format(np.mean(hrt)))
print('qn {}'.format(np.mean(qn)))
print('fif {}'.format(np.mean(fif)))
print('\n')
bad_subs=[]
counter=0
for x in n2:
    if x<11:
        print('bc: {}, counter:{}'.format(x,counter))
        bad_subs.append(counter)
    counter+=1
print('rt')
counter=0
for x in n1:
    if x<10:
        print('bc: {}, counter:{}'.format(x,counter))
    counter+=1

choice_data_clean=np.delete(choice_data_3d,bad_subs,axis=0)
print(len(choice_data_clean))
#np.save('data_hard_V1_V2',choice_data_3d)

rt1 17.303703703703704
rp1 16.992592592592594
cp1 16.91044776119403
ct1 16.705882352941178
n1 17.602941176470587
n2 17.28148148148148
qrt 16.488888888888887
hrt 16.948529411764707
qn 17.22222222222222
fif 9.125925925925927


bc: 6, counter:11
bc: 1, counter:19
bc: 1, counter:23
bc: 10, counter:24
bc: 8, counter:35
bc: 6, counter:39
bc: 0, counter:60
bc: 10, counter:65
bc: 0, counter:75
bc: 3, counter:77
bc: 9, counter:80
bc: 6, counter:99
bc: 7, counter:112
bc: 10, counter:121
bc: 10, counter:125
bc: 10, counter:127
rt
bc: 1, counter:1
bc: 8, counter:11
bc: 8, counter:25
bc: 2, counter:40
bc: 0, counter:41
bc: 3, counter:66
bc: 8, counter:83
bc: 0, counter:110
bc: 9, counter:113
bc: 9, counter:130
bc: 7, counter:132
120


In [103]:

print('rt1 {}'.format(np.mean(rt1)))
print('rp1 {}'.format(np.mean(rp1)))
print('cp1 {}'.format(np.mean(cp1)))
print('ct1 {}'.format(np.mean(ct1)))
print('n1 {}'.format(np.mean(n1)))
print('n2 {}'.format(np.mean(n2)))
print('qrt {}'.format(np.mean(qrt)))
print('hrt {}'.format(np.mean(hrt)))
print('qn {}'.format(np.mean(qn)))
print('fif {}'.format(np.mean(fif)))
print('\n')
bad_subs=[]
counter=0
for x in n2:
    if x<11:
        print('bc: {}, counter:{}'.format(x,counter))
        bad_subs.append(counter)
    counter+=1
print('rt')
counter=0
for x in n1:
    if x<10:
        print('bc: {}, counter:{}'.format(x,counter))
    counter+=1

choice_data_clean=np.delete(choice_data_3d,bad_subs,axis=0)
print(len(choice_data_clean))
#np.save('data_hard_V1_V2',choice_data_3d)


rt1 17.303703703703704
rp1 17.014705882352942
cp1 16.91044776119403
ct1 16.705882352941178
n1 17.602941176470587
n2 17.28148148148148
qrt 16.488888888888887
hrt 16.948529411764707
qn 17.22222222222222
fif 9.125925925925927


bc: 6, counter:11
bc: 1, counter:19
bc: 1, counter:23
bc: 10, counter:24
bc: 8, counter:35
bc: 6, counter:39
bc: 0, counter:60
bc: 10, counter:65
bc: 0, counter:75
bc: 3, counter:77
bc: 9, counter:80
bc: 6, counter:99
bc: 7, counter:112
bc: 10, counter:121
bc: 10, counter:125
bc: 10, counter:127
rt
bc: 1, counter:1
bc: 8, counter:11
bc: 8, counter:25
bc: 2, counter:40
bc: 0, counter:41
bc: 3, counter:66
bc: 8, counter:83
bc: 0, counter:110
bc: 9, counter:113
bc: 9, counter:130
bc: 7, counter:132
120
